#Setup

In [1]:
import pandas as pd
import numpy as np
from pulp import *
from openpyxl import load_workbook

#Data Import

In [17]:
amCourses = pd.read_excel("simple_sample.xlsx", sheetname="AMCourses")
pmCourses = pd.read_excel("simple_sample.xlsx", sheetname="PMCourses")
rooms = pd.read_excel("simple_sample.xlsx", sheetname="Rooms")
courses = pd.read_excel("simple_sample.xlsx", sheetname="Courses")
amPrefs = pd.read_excel("simple_sample.xlsx", sheetname="AMPreferences")
pmPrefs = pd.read_excel("simple_sample.xlsx", sheetname="PMPreferences")

In [14]:
locationPrefs = pd.read_excel("keti.xlsx", sheetname="LocationSetRankingNoZeros")
locationHotels = pd.read_excel("keti.xlsx", sheetname="Hotel Capacity")

#Data Manipulation Functions

In [15]:
def df_to_dict_index(df):
    tmp = df.to_dict(orient = "records")
    return {df.index[i]: tmp[i] for i in range(len(df.index))}

def df_to_dict_values(df):
    d = {}
    for val in {v[i] for v in df.values for i in range(len(v))}:
            for ind in df.index:
                for col in df.columns:
                    if df[col][ind] == val:
                        if val in d.keys():
                            d[val].append(ind)
                        else:
                            d[val] = [ind]
    return d

def dict_to_df_values(d):
    max_len = max([len(d[k]) for k in d.keys()])
    for key in d.keys():
        for i in range(max_len - len(d[key])):
            d[key].append(None)
    return pd.DataFrame(d)

def series_to_dict_listvalues(series):
    d = {}
    for val in {v[i] for v in series.values for i in range(len(v))}:
            for ind in series.index:
                if val in series[ind]:
                    if val in d.keys():
                        d[val].append(ind)
                    else:
                        d[val] = [ind]
    return d

#Optimization Functions

In [19]:
def apply_lp(Preferences, Students, Courses, Rooms, CourseCapacities, RoomCapacities):
    
    assignments = [(i,j) for i in Students for j in Courses]
    course_vars = LpVariable.dicts("Asssignment",assignments,0,1,LpInteger)

    # Create the 'prob' variable to contain the problem data 
    prob = LpProblem("Test Problem",LpMinimize)
    # The objective function is added to 'prob' first 
    prob += lpSum([lpSum([Preferences[i][j]*course_vars[i,j] for j in
    Courses]) for i in Students]), "Total Sum of Preferences for Students"

    # Constraints are added to 'prob'

    # Every student must be assigned a course
    for student in Students:
        prob += lpSum([course_vars[student, i] for i in Courses]) == 1,student + "_Assigned"

    # Course capacity must not be exceeded
    for course in Courses:
        prob += lpSum([course_vars[i,course]] for i in Students) <= CourseCapacities[course],course + "_CourseCapacity"

    # The problem data is written to an .lp file 
    prob.writeLP("Test.lp")
    # The problem is solved using PuLP's choice of Solver 
    prob.solve()
    # The status of the solution is printed to the screen 
    print ("Status:", LpStatus[prob.status])
    # Each of the variables is printed with its resolved optimum value
    for v in prob.variables():    
        print (v.name, "=", v.varValue)
    # The optimised objective function value is printed to the screen    
    print ( "Courses: Total Sum of Assignments = ", value(prob.objective), "\n")
    return prob.objective, prob.variables()

In [19]:
def apply_location_lp(Preferences, Students, Locations, HotelCapacities):
    
    assignments = [(i,j) for i in Students for j in Locations]
    location_vars = LpVariable.dicts("Asssignment",assignments,0,1,LpInteger)

    # Create the 'prob' variable to contain the problem data 
    prob = LpProblem("Location Optimization Problem",LpMinimize)
    # The objective function is added to 'prob' first 
    prob += lpSum([lpSum([Preferences[i][j]*location_vars[i,j] for j in
    Locations]) for i in Students]), "Total Sum of Preferences for Students"

    # Constraints are added to 'prob'

    # Every student must be assigned at least 2 locations
    for student in Students:
        prob += lpSum([location_vars[student, i] for i in Locations]) >= 2,str(student) + "_Assigned"

    # Hotel capacity must not be exceeded
    for location in Locations:
        prob += lpSum([location_vars[i,location]] for i in Students) <= HotelCapacities[location],location + "_HotelCapacity"

    # The problem data is written to an .lp file 
    prob.writeLP("Location.lp")
    # The problem is solved using PuLP's choice of Solver 
    prob.solve()
    # The status of the solution is printed to the screen 
    print ("Status:", LpStatus[prob.status])
    # Each of the variables is printed with its resolved optimum value
    for v in prob.variables():    
        print (v.name, "=", v.varValue)
    # The optimised objective function value is printed to the screen    
    print ( "Courses: Total Sum of Assignments = ", value(prob.objective), "\n")
    return prob.objective, prob.variables()

In [20]:
def run_optimization(dfPrefs, dfCourses, dfRooms):
    Preferences = df_to_dict_index(dfPrefs)
    Students = set(dfPrefs.index)
    Courses = set(dfCourses.index)
    Rooms = dfCourses['Room'].to_dict()
    CourseCapacities = dfCourses['Capacity'].to_dict()
    RoomCapacities = dfRooms['Capacity'].to_dict()
    return apply_lp(Preferences, Students, Courses, Rooms, CourseCapacities, RoomCapacities)

In [20]:
def run_location_optimization(dfLocationPrefs, dfLocations):
    LocationPreferences = df_to_dict_index(dfLocationPrefs)
    Students = set(dfLocationPrefs.index)
    Locations = set(dfLocations.index)
    #Rooms = dfCourses['Room'].to_dict()
    HotelCapacities = dfLocations['Capacity'].to_dict()
    #RoomCapacities = dfRooms['Capacity'].to_dict()
    return apply_location_lp(LocationPreferences, Students, Locations, HotelCapacities)



#Optimization Execution

In [21]:
_, amOutput = run_optimization(amPrefs, amCourses, rooms)
_, pmOutput = run_optimization(pmPrefs, pmCourses, rooms)

Status: Optimal
Asssignment_('Student1',_'Course1') = 1.0
Asssignment_('Student1',_'Course2') = 0.0
Asssignment_('Student1',_'Course3') = 0.0
Asssignment_('Student10',_'Course1') = 0.0
Asssignment_('Student10',_'Course2') = 1.0
Asssignment_('Student10',_'Course3') = 0.0
Asssignment_('Student11',_'Course1') = 1.0
Asssignment_('Student11',_'Course2') = 0.0
Asssignment_('Student11',_'Course3') = 0.0
Asssignment_('Student12',_'Course1') = 0.0
Asssignment_('Student12',_'Course2') = 1.0
Asssignment_('Student12',_'Course3') = 0.0
Asssignment_('Student13',_'Course1') = 0.0
Asssignment_('Student13',_'Course2') = 1.0
Asssignment_('Student13',_'Course3') = 0.0
Asssignment_('Student14',_'Course1') = 1.0
Asssignment_('Student14',_'Course2') = 0.0
Asssignment_('Student14',_'Course3') = 0.0
Asssignment_('Student15',_'Course1') = 0.0
Asssignment_('Student15',_'Course2') = 0.0
Asssignment_('Student15',_'Course3') = 1.0
Asssignment_('Student16',_'Course1') = 0.0
Asssignment_('Student16',_'Course2') = 0.

In [21]:
_, LocationOutput = run_location_optimization(locationPrefs, locationHotels)

{1: {'EW': 3, 'MW': 4, 'HK': 1000, 'MA': 2, 'KR1': 1000, 'EA': 1, 'KS': 1000}, 2: {'EW': 1, 'MW': 1000, 'HK': 1000, 'MA': 1000, 'KR1': 1000, 'EA': 2, 'KS': 1000}, 3: {'EW': 1000, 'MW': 2, 'HK': 1000, 'MA': 1, 'KR1': 1000, 'EA': 1000, 'KS': 1000}, 4: {'EW': 1000, 'MW': 1000, 'HK': 2, 'MA': 1000, 'KR1': 1000, 'EA': 1000, 'KS': 1}, 5: {'EW': 1000, 'MW': 1000, 'HK': 1000, 'MA': 1000, 'KR1': 2, 'EA': 1000, 'KS': 1}, 6: {'EW': 1000, 'MW': 1, 'HK': 2, 'MA': 1000, 'KR1': 1000, 'EA': 1000, 'KS': 1000}, 7: {'EW': 2, 'MW': 1000, 'HK': 1000, 'MA': 1, 'KR1': 1000, 'EA': 1000, 'KS': 1000}, 8: {'EW': 3, 'MW': 1000, 'HK': 1000, 'MA': 2, 'KR1': 1000, 'EA': 1, 'KS': 1000}, 9: {'EW': 2, 'MW': 1, 'HK': 1000, 'MA': 1000, 'KR1': 1000, 'EA': 1000, 'KS': 1000}, 10: {'EW': 1000, 'MW': 4, 'HK': 3, 'MA': 1000, 'KR1': 1, 'EA': 1000, 'KS': 2}, 11: {'EW': 1, 'MW': 2, 'HK': 1000, 'MA': 1000, 'KR1': 1000, 'EA': 1000, 'KS': 1000}, 12: {'EW': 1000, 'MW': 2, 'HK': 1000, 'MA': 1, 'KR1': 1000, 'EA': 1000, 'KS': 1000}, 13:

#Report Generation

In [22]:
amAssignments = pd.Series()
pmAssignments = pd.Series()

for v in amOutput:
    s = v.name.split("\'")
    if v.varValue == 1:
        amAssignments.set_value(s[1], s[3])

for v in pmOutput:
    s = v.name.split("\'")
    if v.varValue == 1:
        pmAssignments.set_value(s[1], s[3])
        
Assignments = pd.DataFrame({'AM Course':amAssignments, 'PM Course':pmAssignments})

In [23]:
Cohorts = dict_to_df_values(df_to_dict_values(Assignments))

In [24]:
notAssigned = pd.Series()

for v in amOutput:
    s = v.name.split("\'")
    if v.varValue == 0 and amPrefs[s[3]][s[1]] < amPrefs[Assignments['AM Course'][s[1]]][s[1]]:
        if s[1] in notAssigned.index:
            notAssigned[s[1]].append(s[3])
        else:
            notAssigned.set_value(s[1], [s[3]])

for v in pmOutput:
    s = v.name.split("\'")
    if v.varValue == 0 and pmPrefs[s[3]][s[1]] < pmPrefs[Assignments['PM Course'][s[1]]][s[1]]:
        if s[1] in notAssigned.index:
            notAssigned[s[1]].append(s[3])
        else:
            notAssigned.set_value(s[1], [s[3]])

In [25]:
WaitLists = dict_to_df_values(series_to_dict_listvalues(notAssigned))

In [26]:
Capacities = pd.DataFrame(amCourses.append(pmCourses)['Capacity'])
Capacities['Occupied Seats'] = pd.Series({k:len(v) for k,v in df_to_dict_values(Assignments).items()})
Capacities['Remaining Seats'] = Capacities['Capacity'] - Capacities['Occupied Seats']

#Output

In [27]:
book = load_workbook('simple_sample.xlsx')
writer = pd.ExcelWriter('simple_sample.xlsx', engine='openpyxl') 
writer.book = book
writer.sheets = dict((ws.title, ws) for ws in book.worksheets)
Assignments.to_excel(writer, sheet_name='Assignments')
Cohorts.to_excel(writer, sheet_name='Cohorts', index=False)
WaitLists.to_excel(writer, sheet_name='WaitLists',index=False)
Capacities.to_excel(writer, sheet_name='Capacities')
writer.save()